In [1]:
import pandas as pd

In [2]:
def get_delta(TotalDist: list, n: int, i: int, j: int):
    return (TotalDist[i] - TotalDist[j]) / (n-2)

In [3]:
def get_matrix_min_pos(matrix: pd.DataFrame):
    min_value = float('inf')
    min_value_pos = (-1, -1)
    for i in matrix.index:
        for j in matrix.columns:
            if i != j and matrix.iloc[i, j] < min_value:
                min_value = matrix.iloc[i, j]
                min_value_pos = (i, j)

    return min_value_pos
    

In [4]:
def get_limb_length_i(matrix: pd.DataFrame, i: int, j: int, delta: float):
    return (1/2) * (matrix.iloc[i, j] + delta)

def get_limb_length_j(matrix: pd.DataFrame, i: int, j: int, delta: float):
    return (1/2) * (matrix.iloc[i, j] - delta)


In [5]:
def calculate_new_D_km(D: pd.DataFrame, k: int, i: int, j: int):
    return (1/2) * (D.iloc[k, i] + D.iloc[k, j] - D.iloc[i, j])

In [ ]:
def get_delta(TotalDist: list, n: int, i: int, j: int):
    return (TotalDist[i] - TotalDist[j]) / (n - 2)

def get_matrix_min_pos(matrix: pd.DataFrame):
    min_value = float('inf')
    min_value_pos = (-1, -1)
    for i in matrix.index:
        for j in matrix.columns:
            if i != j and matrix.loc[i, j] < min_value:
                min_value = matrix.loc[i, j]
                min_value_pos = (i, j)

    return min_value_pos
    
def get_limb_length_i(matrix: pd.DataFrame, i: int, j: int, delta: float):
    return (1/2) * (matrix.loc[i, j] + delta)

def get_limb_length_j(matrix: pd.DataFrame, i: int, j: int, delta: float):
    return (1/2) * (matrix.loc[i, j] - delta)

def calculate_new_D_km(D: pd.DataFrame, k: int, i: int, j: int):
    return (1/2) * (D.loc[k, i] + D.loc[k, j] - D.loc[i, j])

def NeighborJoining(n: int, D: pd.DataFrame, current_m: int) -> dict[int, dict[int, float]]:
    tree = {}
    
    if n == 2:
        tree[D.columns[0]] = {D.columns[1]: D.at[D.columns[0], D.columns[1]]}
        tree[D.columns[1]] = {D.columns[0]: D.at[D.columns[1], D.columns[0]]}
        return tree
    
    TotalDist_D = D.sum().tolist()
    
    D_star = pd.DataFrame(0, index=D.index, columns=D.columns)
    for i in D.index:
        for j in D.columns:
            if i != j:
                D_star.loc[i, j] = (n - 2) * D.loc[i, j] - TotalDist_D[D.index.get_loc(i)] - TotalDist_D[D.columns.get_loc(j)]
    
    i, j = get_matrix_min_pos(D_star)
    
    delta = get_delta(TotalDist_D, n, D.index.get_loc(i), D.index.get_loc(j))
    limb_length_i = get_limb_length_i(D, i, j, delta)
    limb_length_j = get_limb_length_j(D, i, j, delta)

    new_D = D.drop(index=[i, j], columns=[i, j])

    new_m = current_m

    new_row_col = []
    for k in new_D.index:
        D_km = calculate_new_D_km(D, k, i, j)
        new_row_col.append(D_km)
    
    new_D[new_m] = new_row_col
    new_row_col.append(0)
    new_D.loc[new_m] = new_row_col

    D = new_D
    n = len(D)

    tree = NeighborJoining(n, D, current_m + 1)

    if i not in tree:
        tree[i] = {}
    if j not in tree:
        tree[j] = {}
    if new_m not in tree:
        tree[new_m] = {}
    
    tree[new_m][i] = limb_length_i
    tree[new_m][j] = limb_length_j
    tree[i][new_m] = limb_length_i
    tree[j][new_m] = limb_length_j

    # tree[new_m] = {i: limb_length_i, j: limb_length_j}
    return tree


with open('dataset_40204_6.txt', 'r') as file:
    lines = []
    for line in file:
        lines.append(line)

n = int(lines[0].strip())
D = []
for line in lines[1:]:
    row = list(map(int, line.split()))
    D.append(row)

D = pd.DataFrame(D)
D.columns = range(n)
D.index = range(n)

current_m = n

myTree = NeighborJoining(n, D, current_m)

def displayTree(tree):
    for key in sorted(tree.keys()):
        for value in tree[key]:
            print(f'{key}->{value}:{tree[key][value]:.3f}')

displayTree(myTree)

/var/folders/82/5qhfyxm533b7cqcd_05kjlb80000gn/T/ipykernel_54757/2332670508.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-44117.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  D_star.loc[i, j] = (n - 2) * D.loc[i, j] - TotalDist_D[D.index.get_loc(i)] - TotalDist_D[D.columns.get_loc(j)]
/var/folders/82/5qhfyxm533b7cqcd_05kjlb80000gn/T/ipykernel_54757/2332670508.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-36635.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  D_star.loc[i, j] = (n - 2) * D.loc[i, j] - TotalDist_D[D.index.get_loc(i)] - TotalDist_D[D.columns.get_loc(j)]
/var/folders/82/5qhfyxm533b7cqcd_05kjlb80000gn/T/ipykernel_54757/2332670508.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and wi

0->49:630.712
1->33:578.983
2->39:531.174
3->42:595.306
4->38:549.760
5->39:493.826
6->43:552.862
7->32:534.033
8->45:525.537
9->46:606.164
10->40:505.330
11->32:505.967
12->36:533.111
13->47:539.567
14->45:520.463
15->47:498.433
16->49:584.288
17->37:476.930
18->34:459.286
19->35:550.343
20->37:571.070
21->41:529.357
22->46:479.836
23->40:554.670
24->44:549.368
25->42:460.694
26->34:569.714
27->38:499.240
28->43:473.138
29->33:482.017
30->41:592.643
31->35:525.657
32->48:121.496
32->7:534.033
32->11:505.967
33->48:112.004
33->1:578.983
33->29:482.017
34->53:197.674
34->18:459.286
34->26:569.714
35->36:26.889
35->19:550.343
35->31:525.657
36->54:207.086
36->12:533.111
36->35:26.889
37->44:94.132
37->17:476.930
37->20:571.070
38->52:177.106
38->4:549.760
38->27:499.240
39->53:176.576
39->2:531.174
39->5:493.826
40->56:172.359
40->10:505.330
40->23:554.670
41->51:118.142
41->21:529.357
41->30:592.643
42->50:106.911
42->3:595.306
42->25:460.694
43->50:107.339
43->6:552.862
43->28:473.138


/var/folders/82/5qhfyxm533b7cqcd_05kjlb80000gn/T/ipykernel_54757/2332670508.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-7267.6875' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  D_star.loc[i, j] = (n - 2) * D.loc[i, j] - TotalDist_D[D.index.get_loc(i)] - TotalDist_D[D.columns.get_loc(j)]
/var/folders/82/5qhfyxm533b7cqcd_05kjlb80000gn/T/ipykernel_54757/2332670508.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-6762.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  D_star.loc[i, j] = (n - 2) * D.loc[i, j] - TotalDist_D[D.index.get_loc(i)] - TotalDist_D[D.columns.get_loc(j)]
/var/folders/82/5qhfyxm533b7cqcd_05kjlb80000gn/T/ipykernel_54757/2332670508.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and w